In [71]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import pandas as pd
import math
import numpy as np
import os
import time 

import cntk as C
import cntk.tests.test_utils
from cntk.layers import *
from cntk.layers.typing import *
import pickle
import random
from cntk import sequence
from cntk import load_model
from cntk.device import try_set_default_device, gpu,cpu
from scipy.sparse import csr_matrix

from gensim.models import Word2Vec
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components
try_set_default_device(gpu(0))


vocab_size = 80000
num_labels = 19#19
title_size = 52000
body_size  = 210000
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200

max_length_title = 53
max_length_body  = 200

suffix = "180days_all_shuffled"
#suffix = "linkedin_only"
prefix = "/home/t-haohu/IndustryClassifier/Data/"

#data_token_body        = "{}/middle/{}_token_body.txt".format(prefix,suffix)
data_train_sample = "{}/middle/train_{}.txt".format(prefix,suffix)
#data_train_sample = "{}/middle/train_{}_with_linkedin_all.txt".format(prefix,suffix)
data_test_sample  = "{}/middle/test_{}.txt".format(prefix,suffix)
#data_test_sample_editor  = "{}/middle/test_{}_editor.txt".format(prefix,suffix)

data_title_sample    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_sample     = "{}/ready/body_{}.wl".format(prefix,suffix)
suffix = "180days_all_shuffled"
data_industry_sample = "{}/ready/industry_{}.wl".format(prefix,suffix)
filter_num=200 
dropout_rate = 0.5
emb_dim =300

def load_data(input_file,title_dict,industry_dict):
    data = open(input_file, encoding = "utf-8").readlines()
    
    data_title =np.zeros((len(data),max_length_title),dtype = np.float32)
    data_label = np.zeros((len(data),1),dtype = np.float32)
       
    for index,line in enumerate(data):
        row = line.strip("\n").split("\t")       
        title    =  row[0]
        industry =  row[1]
        
        for jndex,token in enumerate(title.split(" ")):
            if jndex>=max_length_title:
                break
            data_title[index,jndex]=title_dict.get(token,len(title_dict)-1)    
        data_label[index] = industry_dict.get(industry,len(industry_dict))
    return data_title,data_label


def load_embedding(title_file,embedding_model_file):
    model = Word2Vec.load(embedding_model_file)
    title_list = [x.strip("\n") for x in open(title_file,encoding = 'utf-8').readlines()]
    embedding = np.zeros((len(title_list),emb_dim))
    count = 0
    for i,w in enumerate(title_list):
        try:
            vec = model.wv[w]
        except:
            vec=model.wv["UNK"]
            count+=1
        embedding[i] =vec
    print(count)
    return embedding

def get_context_left(current,previous,w_l,w_ls):
    left_c = current@w_l
    left_e = previous@w_ls
    left_h=left_c+left_e
    return C.tanh(left_h)
def get_context_right(current,after,w_r,w_rs):
    right_c = current@w_r
    right_e = after@w_rs
    right_h =right_c+right_e
    return C.tanh(right_h)

def create_model_rcnn(embed = False):
    first_word = C.parameter(shape=(emb_dim))
    last_word = C.parameter(shape=(emb_dim))
    w_l,w_ls,w_r,w_rs = C.parameter(shape=(emb_dim,emb_dim)),C.parameter(shape=(emb_dim,emb_dim)),C.parameter(shape=(emb_dim,emb_dim)),C.parameter(shape=(emb_dim,emb_dim))
    #version 2 : 1 dense layer version3: sigmoid activation in dense
    if embed:
        h1= C.layers.Embedding(weights=embedding,name='embed_1')(input_xt_one_hot)#
    else:
        h1= C.layers.Embedding(emb_dim,name='embed_2')(input_xt_one_hot)#init=embedding,
    previous = first_word
    # h1 [batch*sentence_length*emb_dim]
    context_left_list = []
    for i in range(max_length_title):
        current = C.squeeze(h1[i])
        context_left_list.append(get_context_left(current,previous,w_l,w_ls))
        previous = current
        
    context_right_list = []
    after = last_word
    for i in reversed(range(max_length_title)):
        current = C.squeeze(h1[i])
        context_right_list.append(get_context_right(current,after,w_r,w_rs))
        after = current
    total_list = []
    for i in range(max_length_title):
        total_list.append(C.splice(h1[i],context_left_list[i],context_right_list[i]))
    #output = C.splice(total_list)
    #print(output)
    #h2=BiRecurrence(C.layers.LSTM(hidden_dim), C.layers.LSTM(hidden_dim))(h1)
    h3=C.element_max(*total_list)
    print(h3)
    drop1 = C.layers.Dropout(dropout_rate)(h3)
    h4=C.layers.Dense(num_labels,name='hidden')(drop1)

    return h4


def batch_iter(data,batch_size, num_epochs, shuffle=True):
    # Generates a batch iterator for a dataset.
    data_size = len(data)
    num_batches_per_epoch = int((data_size-1)/batch_size) + 1
    print('data_size: ', data_size, 'batch_size: ', batch_size, 'num_batches_per_epoch: ', num_batches_per_epoch)
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            random.shuffle(data)
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield data[start_index:end_index]
            

def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_sample).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_sample).readlines()])}




input_xt = C.input_variable(shape=(max_length_title),  dtype=np.float32)
#input_xt = C.input_variable(**Sequence[Tensor[1]])
input_y  = C.input_variable(shape=(1)               ,  dtype=np.int)

input_xt_one_hot = C.one_hot(input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
input_y_one_hot = C.one_hot(input_y  , num_classes=len(industry_dict) ,  sparse_output=True)


test_data  = load_data(data_test_sample,title_dict,industry_dict)
train_data = load_data(data_train_sample,title_dict,industry_dict)
#test_data_editor  = load_data(data_test_sample_editor,title_dict,industry_dict)
embedding = load_embedding(data_title_sample,"word2vec.model")

def test(batch_size,model,data):
    scores = model(input_xt)
    predict = C.argmax(scores,axis = 0)
    confuse = np.zeros((num_labels,num_labels))

    test_data_title,test_data_label = data
    batches = batch_iter(list(zip(test_data_title,test_data_label)), batch_size, 1)
    
    for batch in batches:
        batch_data_title,batch_data_label = zip(*batch) 
        output = np.array(predict.eval({input_xt: np.array(batch_data_title)}),dtype=np.int)
        gt = np.array(batch_data_label,dtype=np.int)
        confuse+=fast_hist(output,gt,num_labels)
        
    precision=np.diag(confuse)/np.sum(confuse,axis=0)
    recall = np.diag(confuse)/np.sum(confuse,axis=1)
    accuarcy = np.diag(confuse).sum() / confuse.sum()
    aver_precision=np.nanmean(precision)
    aver_recall = np.nanmean(recall)
   
    print("Precision:{} Recall:{} Acc:{}".format(aver_precision,aver_recall,accuarcy))
    return accuarcy
def train(train_data,num_epochs,learning_rate,batch_size,tag="CNN",l2_weight=0):
    global model
    #learning_rate *= batch_size
    model = create_model_rcnn()
    print(C.logging.get_node_outputs(model))
    scores = model(input_xt)

    loss =C.reduce_mean(C.losses.cross_entropy_with_softmax(scores, input_y_one_hot))
    
    # Training
    lr_schedule = C.learning_parameter_schedule(learning_rate)
    #learner = C.adam(scores.parameters, lr=lr_schedule, momentum=0.9,l2_regularization_weight=0)
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=num_epochs)
    momentums = C.momentum_schedule(0.99, minibatch_size=batch_size)
    learner = C.adam(parameters=scores.parameters,#model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15,
                     gradient_clipping_with_truncation=True,
                     l2_regularization_weight=l2_weight)
    trainer = C.Trainer(scores, (loss), [learner], progress_printer)
    
    train_data_title,train_data_label = train_data
    batches = batch_iter(list(zip(train_data_title,train_data_label)), batch_size, num_epochs)

    # training loop
    count = 0
    t = time.time()
    for batch in batches:
        count += 1
        batch_data_title,batch_data_label = zip(*batch)
        batch_data_title = list(batch_data_title)
        #print(type(batch_data_title),type(batch_data_title[0]),batch_data_title[0])
        trainer.train_minibatch({input_xt: np.array(batch_data_title), input_y: np.array(batch_data_label)})
        if count%1000== 0:
            print(count,time.time()-t)
            t=time.time()
            acc1=test(batch_size,model,test_data)
            #acc2=test(batch_size,model,test_data_editor)
            
            # save model
            #model.save('./model/{}/{}_acc{:.3f}.dnn'.format(suffix,tag,acc1))
            #model.save('./model/{}/{}_acc1{:.3f}_acc2{:.3f}.dnn'.format(suffix,tag,acc1,acc2))
    

    # save model
    


    

0


In [ ]:
train(train_data,num_epochs=20,learning_rate=[5e-3*150]*2+[1e-4*150],batch_size = 150,tag = "RCNN")

Composite(Tensor[53]) -> Tensor[1,900]
[Output('hidden', [#], [19]), Output('Block1064081_Output_0', [#], [1 x 900]), Output('Block1064056_Output_0', [#], [1 x 900]), Output('Block1063624_Output_0', [#], [1 x 900]), Output('Block1063416_Output_0', [#], [1 x 900]), Output('Block1063304_Output_0', [#], [1 x 900]), Output('Block1063256_Output_0', [#], [1 x 900]), Output('Splice1062917_Output_0', [#], [1 x 900]), Output('Slice1062914_Output_0', [#], [1 x 300]), Output('embed_2', [#], [53 x 300]), Output('OneHotOp685414_Output_0', [#], [53 x 56178]), Output('Tanh1061021_Output_0', [#], [300]), Output('Plus1061018_Output_0', [#], [300]), Output('Times1061012_Output_0', [#], [300]), Output('Squeeze1061009_Output_0', [#], [300]), Output('Slice1061006_Output_0', [#], [1 x 300]), Output('Times1061015_Output_0', [], [300]), Output('Tanh1061975_Output_0', [#], [300]), Output('Plus1061972_Output_0', [#], [300]), Output('Times1061966_Output_0', [#], [300]), Output('Squeeze1061963_Output_0', [#], [30

data_size:  768505 batch_size:  150 num_batches_per_epoch:  5124
Learning rate per minibatch: 0.75
1000 190.03798484802246
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7075636102112516 Recall:0.745998784561532 Acc:0.792342725285402
2000 208.41464233398438
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7290602585486484 Recall:0.7717185176565209 Acc:0.8106297060966723
3000 252.63176703453064
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7339414647498458 Recall:0.7809902631373381 Acc:0.8128916686907943
4000 257.382417678833
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7412094112054843 Recall:0.7767064579870131 Acc:0.8163195287830944
5000 221.46359944343567
data_size:  329360 batch_size:  150 num_batches_per_epoch:  2196
Precision:0.7550281739905812 Recall:0.7752285630115681 Acc:0.8240253825601166
6000 236.656672000885
data_size:  329360 batch_size:  150 num_batches_

In [9]:
import cntk as C
a =C.input_variable(shape=(10,10))
b =C.input_variable(shape=(10,10))
c= C.combine(a,b)
print(c[0])
for i,x in enumerate(a):
    print(x)
    if i == 9:
        break
print("233",a[0])

Input('Input233', [#], [10 x 10])
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
Slice(Tensor[10,10]) -> Tensor[1,10]
233 Slice(Tensor[10,10]) -> Tensor[1,10]


In [43]:
from cntk.layers import *
from cntk.layers.typing import *
a =C.input_variable(**Sequence[Tensor[1]])
b= [[[1],[3],[5]],[[1],[3],[5],[7]]]
c = C.plus(a,a)
d = C.layers.Fold(C.element_max)(c)
w = C.parameter(shape=(1))
e = d*w
learner = C.adam(parameters=e.parameters,#model.parameters,
                     lr=1e-3,
                     momentum=0.9,
                     gradient_clipping_threshold_per_sample=15,
                     gradient_clipping_with_truncation=True,
                     l2_regularization_weight=0)
trainer = C.Trainer(e, (e), [learner])
print(e(b))
trainer.train_minibatch({a:b})

[[0.]
 [0.]]


True

In [67]:
a = C.input_variable(shape=(2,2))
b = C.plus(a,a)
d = b+b
c = C.element_max(*[a,b,d])
c([[[1,2],[3,4]]])

array([[[ 4.,  8.],
        [12., 16.]]], dtype=float32)